# Causal Reasoning with Vector-HASH and LLM

本 notebook 实现基于 Vector-HASH 模型的因果推理评估：
1. 加载 NV-Embed-v2 模型
2. 构建 INDEX → 最大 Cluster 映射
3. Vector-HASH 推理评估（与 Gemma 答案对比）
4. Vector-HASH 重建率分析

## 工作流程说明
对于每个样本，按以下步骤进行 Vector-HASH 推理：

$$s_i \rightarrow p_i \rightarrow g_i^{noisy} \rightarrow g_i^{clean} \Rightarrow g_{i+1}^{clean} \rightarrow p_{i+1} \rightarrow s_{i+1}$$

- 步骤1: $s_i$ 通过 $W_{sp}$ 映射到 $p_i$
- 步骤2: $p_i$ 通过 $W_{gp}$ 映射到 $g_i^{noisy}$
- 步骤3: 对 $g_i^{noisy}$ 进行模块化最近邻清洗得到 $g_i^{clean}$
- 步骤4: 根据 askfor 确定方向，通过 $W_{gg}$ 进行路径积分得到 $g_{i+1}^{clean}$
- 步骤5: $g_{i+1}^{clean}$ 通过 $W_{pg}$ 映射到 $p_{i+1}$
- 步骤6: $p_{i+1}$ 通过 $W_{ps}$ 映射回 sensory 空间得到 $\hat{s}_{i+1}$

**注意**：推理流程中不进行归一化，$g_i^{clean}$ 与理论上应该对应的向量直接作对比

## Section 1: 环境配置和 NVembed 加载

In [ ]:
# ===== Cell 1: 导入依赖库和加载 NV-Embed-v2 模型 =====
import os
import sys
import json
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# 添加 src 目录到 path
sys.path.insert(0, '/home/amax/Zixing_Jia/Vector-HASH-tinkering-main/src')

# ===== 配置路径 =====
BASE_DIR = "/home/amax/Zixing_Jia/Vector-HASH-tinkering-main"
DATA_DIR = os.path.join(BASE_DIR, "Gemma_anwer_causal/Causal_Data")
RESULTS_DIR = os.path.join(BASE_DIR, "Gemma_anwer_causal/Causal_Results")
JSONL_PATH = os.path.join(BASE_DIR, "Ecare/train.jsonl")

os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"Base dir: {BASE_DIR}")
print(f"Data dir: {DATA_DIR}")
print(f"Results dir: {RESULTS_DIR}")

# ===== 加载 NV-Embed 模型 =====
print("\n" + "="*60)
print("Loading NV-Embed-v2 Model...")
print("="*60)

from transformers import AutoModel, AutoTokenizer

model_path = "/home/amax/Gemma_and_NVembed/NV/snapshots/main"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, local_files_only=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True, local_files_only=True, torch_dtype=torch.float16)
model = model.to(device)
model.eval()

print("✅ NV-Embed model loaded successfully!")

In [ ]:
# ===== Cell 2: 定义编码函数（支持 batch_size 配置）=====
"""支持可配置 batch_size 的文本编码函数"""

def encode_texts(texts, max_length=4096, batch_size=32, instruction=""):
    """
    批量编码文本为向量
    
    Args:
        texts: 单个字符串或字符串列表
        max_length: 最大序列长度
        batch_size: 批处理大小（可配置）
        instruction: 可选的指令前缀
    
    Returns:
        numpy array of shape (n_texts, 4096)
    """
    if isinstance(texts, str):
        texts = [texts]
    
    eos_token = tokenizer.eos_token if tokenizer.eos_token else "</s>"
    
    # 如果有指令，添加指令前缀
    if instruction:
        texts = [f"Instruct: {instruction}\nQuery: {text}" for text in texts]
    
    # 添加 EOS token
    texts_with_eos = [text + eos_token for text in texts]
    
    all_embeddings = []
    
    # 按 batch_size 处理
    for i in range(0, len(texts_with_eos), batch_size):
        batch_texts = texts_with_eos[i:i+batch_size]
        
        batch_dict = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        batch_dict = {k: v.to(device) for k, v in batch_dict.items()}
        
        # 创建 pool_mask
        attention_mask = batch_dict["attention_mask"]
        seq_lengths = attention_mask.sum(dim=1)
        pool_mask = torch.zeros_like(attention_mask)
        for j, length in enumerate(seq_lengths):
            pool_mask[j, length - 1] = 1
        
        with torch.no_grad():
            outputs = model(
                input_ids=batch_dict["input_ids"],
                attention_mask=attention_mask,
                pool_mask=pool_mask
            )
            
            if isinstance(outputs, dict):
                if "sentence_embeddings" in outputs:
                    embeddings = outputs["sentence_embeddings"]
                elif "last_hidden_state" in outputs:
                    embeddings = outputs["last_hidden_state"][:, -1, :]
                else:
                    raise ValueError(f"Cannot extract embeddings: {outputs.keys()}")
            elif hasattr(outputs, "sentence_embeddings"):
                embeddings = outputs.sentence_embeddings
            elif hasattr(outputs, "last_hidden_state"):
                embeddings = outputs.last_hidden_state[:, -1, :]
            else:
                embeddings = outputs
        
        # 归一化
        embeddings = F.normalize(embeddings, p=2, dim=1)
        embeddings_cpu = embeddings.float().cpu()
        all_embeddings.append(np.array(embeddings_cpu.tolist()))
    
    return np.vstack(all_embeddings)

def cosine_similarity(a, b):
    """计算余弦相似度"""
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0.0
    return np.dot(a, b) / (norm_a * norm_b)

print("✅ Helper functions defined!")

## Section 2: 加载因果数据

In [ ]:
# ===== Cell 3: 加载因果数据 =====
print("="*60)
print("Loading Causal Data...")
print("="*60)

# 加载压缩后的因果向量数据
causal_A = np.load(os.path.join(DATA_DIR, 'causal_A_vectors_compressed.npy'))
causal_B = np.load(os.path.join(DATA_DIR, 'causal_B_vectors_compressed.npy'))

with open(os.path.join(DATA_DIR, 'causal_metadata_compressed.json'), 'r', encoding='utf-8') as f:
    causal_metadata = json.load(f)

print(f"✅ causal_A shape: {causal_A.shape}")
print(f"✅ causal_B shape: {causal_B.shape}")
print(f"✅ causal_metadata count: {len(causal_metadata)}")

# 统计 askfor 分布
askfor_counts = defaultdict(int)
for meta in causal_metadata:
    askfor_counts[meta['ASKFOR']] += 1
print(f"📊 ASKFOR distribution: {dict(askfor_counts)}")

# 加载 E-care/Ecare 数据获取 H1/H2
print("\nLoading Ecare data from train.jsonl...")
e_care_data = {}
with open(JSONL_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line.strip())
        e_care_data[item['index']] = {
            'premise': item['premise'],
            'hypothesis1': item['hypothesis1'],
            'hypothesis2': item['hypothesis2'],
            'label': item['label'],
            'ask-for': item['ask-for']
        }

print(f"✅ Loaded {len(e_care_data)} samples")

## Section 3: 构建 INDEX → 最大 Cluster 映射

这是重建率分析的关键步骤。为了获取每个 INDEX 对应的最大 cluster（样本数最多）的 centroid 向量。

In [ ]:
# ===== Cell 4: 构建 INDEX → 最大 Cluster 映射 =====
"""
目的：构建 INDEX -> 最大 cluster 对应的 sample_idx 的映射

数据结构关系：
- causal_metadata[sample_idx] 包含 {'ASKFOR', 'INDEX', 'cluster_id', 'premise'}
- 同一个 INDEX 可能对应多个 sample_idx（不同 cluster）
- 我们需要找到每个 INDEX 中样本数最多的 cluster
"""

print("="*60)
print("Building INDEX -> Largest Cluster Mapping")
print("="*60)

# Step 1: 按 INDEX 分组 sample_idx
index_to_samples = defaultdict(list)
for i, meta in enumerate(causal_metadata):
    index_to_samples[meta['INDEX']].append({
        'sample_idx': i,
        'cluster_id': meta.get('cluster_id', None),
        'askfor': meta['ASKFOR']
    })

print(f"✅ Found {len(index_to_samples)} unique INDEX")

# Step 2: 加载 clustering_summary 获取每个 cluster 的样本数
CLUSTERING_SUMMARY_PATH = os.path.join(DATA_DIR, "clustering_summary_compressed.json")
with open(CLUSTERING_SUMMARY_PATH, 'r', encoding='utf-8') as f:
    clustering_summary = json.load(f)

print(f"✅ Loaded clustering_summary with {len(clustering_summary)} premises")

# Step 3: 为每个 INDEX 找到最大 cluster
index_to_largest_cluster = {}
for premise, summary in clustering_summary.items():
    idx = summary['index']
    clusters = summary['clusters']
    if clusters:
        # 找到 count 最大的 cluster
        largest_cid = max(clusters.keys(), key=lambda cid: clusters[cid]['count'])
        largest_count = clusters[largest_cid]['count']
        index_to_largest_cluster[idx] = {
            'premise': premise[:50] + '...' if len(premise) > 50 else premise,
            'largest_cluster_id': int(largest_cid),
            'largest_count': largest_count,
            'ask_for': summary['ask_for']
        }

print(f"✅ Found largest cluster info for {len(index_to_largest_cluster)} INDEX")

# Step 4: 构建 INDEX -> 最大 cluster 对应的 sample_idx 的映射
index_to_largest_sample = {}
for idx, samples in index_to_samples.items():
    if idx in index_to_largest_cluster:
        largest_cid = index_to_largest_cluster[idx]['largest_cluster_id']
        # 找到这个 cluster_id 对应的第一个 sample_idx
        for s in samples:
            if s['cluster_id'] == largest_cid:
                index_to_largest_sample[idx] = s['sample_idx']
                break
        # 如果没找到精确匹配，使用第一个
        if idx not in index_to_largest_sample:
            index_to_largest_sample[idx] = samples[0]['sample_idx']

print(f"✅ Built index_to_largest_sample mapping with {len(index_to_largest_sample)} entries")

# 显示几个示例
print("\n📌 Sample mappings:")
for i, (idx, sample_idx) in enumerate(list(index_to_largest_sample.items())[:3]):
    askfor = causal_metadata[sample_idx]['ASKFOR']
    print(f"  INDEX={idx} -> sample_idx={sample_idx}, askfor={askfor}")

## Section 4: 生成候选答案 H1/H2 的 Embeddings

In [ ]:
# ===== Cell 5: 生成 H1/H2 Embeddings =====
"""为每个 E-care 样本生成 hypothesis1 和 hypothesis2 的向量嵌入"""

print("="*60)
print("Generating H1/H2 Embeddings...")
print("="*60)

# 获取需要处理的 INDEX 列表
indices_to_process = list(index_to_largest_sample.keys())
print(f"Processing {len(indices_to_process)} indices...")

# 收集所有需要编码的文本
h1_texts = []
h2_texts = []
valid_indices = []

for idx in indices_to_process:
    if idx in e_care_data:
        h1_texts.append(e_care_data[idx]['hypothesis1'])
        h2_texts.append(e_care_data[idx]['hypothesis2'])
        valid_indices.append(idx)

print(f"✅ Found {len(valid_indices)} valid indices with E-care data")

# 批量编码 H1（使用 batch_size=32）
print("\nEncoding H1 hypotheses...")
h1_embeddings = encode_texts(h1_texts, batch_size=32)
print(f"✅ H1 embeddings shape: {h1_embeddings.shape}")

# 批量编码 H2
print("Encoding H2 hypotheses...")
h2_embeddings = encode_texts(h2_texts, batch_size=32)
print(f"✅ H2 embeddings shape: {h2_embeddings.shape}")

# 存储到 e_care_data
for i, idx in enumerate(valid_indices):
    e_care_data[idx]['h1_vec'] = h1_embeddings[i]
    e_care_data[idx]['h2_vec'] = h2_embeddings[i]

print(f"\n✅ H1/H2 embeddings generated and stored for {len(valid_indices)} samples")

## Section 5: RRM (Vector-HASH) 模型定义

定义 RRM 模型用于因果推理，包含多种采样模式。

In [ ]:
# ===== Cell 6: 定义 RRM 模型 =====
"""RRM (Relational Reasoning Model) - Vector-HASH 核心"""

class RRM_Model:
    """Relevance-Rate Matrix Model for Vector-HASH"""
    def __init__(self, max_effects=7, learning_rate=0.05, decay=0.001, T=1.0):
        """
        Initialize RRM model
        
        Args:
            max_effects: 最大 effect 数量（slots）
            learning_rate: 学习率
            decay: 衰减系数
            T: Temperature for Boltzmann sampling
        """
        self.max_effects = max_effects
        self.eta = learning_rate
        self.lam = decay
        self.T = T
        self.weights = {}  # 存储每个 (source_id, slot) 的权重
        self.counts = {}   # 存储计数
    
    def get_weights(self, key):
        """为给定的 key 获取权重向量"""
        if key not in self.weights:
            self.weights[key] = np.zeros(self.max_effects)
            self.counts[key] = np.zeros(self.max_effects)
        return self.weights[key]
    
    def update(self, key, slot_idx, x_c=1.0, y_o=1.0):
        """更新权重 (Hebbian learning)"""
        w = self.get_weights(key)
        y_vec = np.zeros(self.max_effects)
        y_vec[slot_idx] = y_o
        delta_w = self.eta * x_c * y_vec - self.lam * w
        self.weights[key] += delta_w
        self.weights[key] = np.maximum(self.weights[key], 0)
        self.counts[key][slot_idx] += 1
    
    def sample(self, key, mode="direct"):
        """
        从 RRM 中采样一个 slot
        
        Args:
            key: 源事件的 ID
            mode: 采样模式
                - "direct": 按权重归一化后的概率采样
                - "Boltzmann": 使用 Boltzmann 分布（softmax）采样
                - "max_prob": 直接选择概率最大的 slot（确定性选择）
        
        Returns:
            (sampled_slot, probs): 采样到的 slot 索引和概率分布
        """
        w = self.get_weights(key)
        
        if mode == "direct":
            total_w = np.sum(w)
            if total_w == 0:
                probs = np.ones(self.max_effects) / self.max_effects
            else:
                probs = w / total_w
            return np.random.choice(self.max_effects, p=probs), probs
        
        elif mode == "Boltzmann":
            logits = w / self.T
            exp_logits = np.exp(logits - np.max(logits))
            probs = exp_logits / np.sum(exp_logits)
            return np.random.choice(self.max_effects, p=probs), probs
        
        elif mode == "max_prob":
            # 计算概率分布（用于返回）
            total_w = np.sum(w)
            if total_w == 0:
                probs = np.ones(self.max_effects) / self.max_effects
            else:
                probs = w / total_w
            # 直接选择概率最大的 slot（确定性）
            max_slot = np.argmax(probs)
            return max_slot, probs
        
        else:
            raise ValueError(f"Unknown sampling mode: {mode}")

print("✅ RRM_Model class defined!")

## Section 4.5: 加载 Vector-HASH 模型和权重矩阵

从磁盘加载预训练的权重矩阵 (Wsp, Wps, Wgp, Wpg) 和初始化模型参数。

In [ ]:
# ===== Cell 4.5: 加载 Vector-HASH 模型和权重矩阵 =====
"""
初始化 Vector-HASH 模型参数和加载预训练权重
"""

from src.assoc_utils_np_2D import gen_gbook_2d, path_integration_Wgg_2d, module_wise_NN_2d
from src import assoc_utils_np as assoc_utils

print("="*60)
print("Initialize Vector-HASH Model")
print("="*60)

# ===== 模型超参数 =====
lambdas = [3, 4, 5, 7]
Np = 4096  # Place cells
Ns = 4096  # Sensory cells (NV-Embed dimension)
Ng = int(np.sum(np.square(lambdas)))  # Total grid cells
Npos = int(np.prod(lambdas))  # Grid positions per side
thresh = 2.0  # Nonlinearity threshold
max_effects = 7  # RRM max effects

print(f"Model parameters:")
print(f"  lambdas: {lambdas}")
print(f"  Ng (total grid cells): {Ng}")
print(f"  Npos (position dimension): {Npos}")
print(f"  Np (place cells): {Np}")
print(f"  Ns (sensory dim): {Ns}")
print(f"  thresh: {thresh}")

# ===== 生成 Grid Codebook =====
print(f"\nGenerating grid codebook...")
gbook = gen_gbook_2d(lambdas, Ng, Npos)  # (Ng, Npos, Npos)
gbook_flat = gbook.reshape(Ng, Npos * Npos)
print(f"  gbook shape: {gbook.shape}")
print(f"  gbook_flat shape: {gbook_flat.shape}")

# ===== Module 信息 =====
module_sizes = np.square(lambdas)  # [9, 16, 25, 49]
module_gbooks = [np.eye(int(size)) for size in module_sizes]
print(f"  module_sizes: {module_sizes}")
print(f"  module_gbooks lengths: {[mg.shape for mg in module_gbooks]}")

# ===== 加载/初始化权重矩阵 =====
print(f"\nLoading/initializing weight matrices...")

# 尝试从磁盘加载
Wsp_path = os.path.join(RESULTS_DIR, "Wsp.npy")
Wps_path = os.path.join(RESULTS_DIR, "Wps.npy")
Wpg_path = os.path.join(RESULTS_DIR, "Wpg.npy")
Wgp_path = os.path.join(RESULTS_DIR, "Wgp.npy")
pbook_path = os.path.join(RESULTS_DIR, "pbook.npy")

weights_loaded = True

if os.path.exists(Wsp_path) and os.path.exists(Wpg_path):
    Wsp = np.load(Wsp_path)
    Wps = np.load(Wps_path)
    Wpg = np.load(Wpg_path)
    Wgp = np.load(Wgp_path)
    pbook_flat = np.load(pbook_path)  # shape: (nruns, Np, Npos^2)
    print(f"✅ Loaded weights from {RESULTS_DIR}")

# 处理 pbook: 从 (nruns, Np, Npos^2) reshape 成 (Np, Npos, Npos)
# 这样可以用 pbook[:, x, y] 直接索引
if pbook_flat.ndim == 3 and pbook_flat.shape[0] == 1:
    # pbook_flat shape: (1, Np, Npos^2) -> (Np, Npos, Npos)
    pbook = pbook_flat[0].reshape(Np, Npos, Npos)
else:
    pbook = pbook_flat.reshape(Np, Npos, Npos)
print(f"✅ pbook reshaped: {pbook_flat.shape} -> {pbook.shape}")

Wpg = np.squeeze(Wpg)
print(f"\n   最终 shape:")
print(f"   Wsp: {Wsp.shape}")
print(f"   Wps: {Wps.shape}")
print(f"   Wpg: {Wpg.shape}")
print(f"   Wgp: {Wgp.shape}")
print(f"   pbook: {pbook.shape}")
# ===== 生成 Wgg (路径积分矩阵) =====
print(f"\nGenerating path integration matrices...")
Wgg_c2e = path_integration_Wgg_2d(lambdas, Ng, axis=0, direction=1)   # cause->effect: x+1
Wgg_e2c = path_integration_Wgg_2d(lambdas, Ng, axis=0, direction=-1)  # effect->cause: x-1
print(f"  Wgg_c2e shape: {Wgg_c2e.shape}")
print(f"  Wgg_e2c shape: {Wgg_e2c.shape}")

print("\n✅ Vector-HASH model initialized!")


In [ ]:
# ===== Cell 4.6: 定义 Vector-HASH 辅助函数和推理管道 =====
"""
实现 Vector-HASH 推理的分步函数和完整流程
"""

def normalize_vec(v):
    """归一化向量"""
    norm = np.linalg.norm(v)
    if norm < 1e-12:
        return v
    return v / norm

def sensory_to_place(s, Wps):
    """Sensory -> Place cell (通过 Wsp)"""
    # s: (Ns,), Wsp: (Ns, Np) -> p: (Np,)
    p = Wps @ s  # 注意：s^T Wsp
    return p

def place_to_sensory(p, Wsp):
    """Place cell -> Sensory (通过 Wps)"""
    # p: (Np,), Wps: (Ns, Np) -> s: (Ns,)
    s = Wsp @ p # p^T Wps^T = p^T @ Wps^T
    return s

def place_to_grid_noisy(p, Wgp):
    """Place cell -> Grid cell noisy (通过 Wgp, 无非线性)"""
    # p: (Np,), Wgp: (Ng, Np) -> g: (Ng,)
    g = np.dot(Wgp, p)  # Wgp @ p
    return g

def grid_to_place_with_nonlin(g, Wpg, thresh=2.0):
    """
    Grid cell -> Place cell (通过 Wpg + nonlinearity)
    
    使用 pbook (place cell book) 进行非线性变换
    pbook shape: (Np, Npos, Npos) 或 (Np, Npos*Npos)
    """
    # 首先通过 Wpg 进行线性变换
    # g: (Ng,), Wpg: (Np, Ng) -> p_linear: (Np,)
    p_linear = np.dot(Wpg, g)
    
   
    # 非线性函数（阈值函数）
    # p_output = ReLU(p_linear) 如果超过阈值
    p_output = np.maximum(0, p_linear - thresh)
    
    return p_output

def get_vertical_shift(slot):
    """根据 slot 索引计算 y 方向偏移"""
    # 简单映射：slot -> delta_y
    # slot 在 [0, max_effects-1] 之间
    # 映射到 y 偏移：最多上下移动 k = 3
    k = 3
    delta_y = slot - k  # [-3, -2, -1, 0, 1, 2, 3]
    return delta_y

def vector_hash_infer_with_rrm(s_input, askfor, Wps, Wsp, Wgp, Wpg, 
                                Wgg_c2e, Wgg_e2c, 
                                module_gbooks, module_sizes,
                                rrm_model, source_id, 
                                sampling_mode="direct",
                                statistic=False):
    """
    完整的 Vector-HASH 推理流程（含RRM采样）:
    
    推理流程（参考Causal_V2）:
    1. s_i -> p_i (通过 Wsp)
    2. p_i -> g_i^noisy (通过 Wgp)
    3. g_i^noisy -> g_i^clean (通过 module_wise_NN_2d吸引子动力学)
    4. 从RRM采样一个slot，决定y方向偏移
    5. Path Integration: 
       - x方向: 由askfor决定 (cause->effect: x+1, effect->cause: x-1)
       - y方向: 由采样的slot决定
    6. g_{i+1}^clean -> p_{i+1} (通过 Wpg + nonlin) 
       注意：path_integration后g已经是clean的，无需再清洗
    7. p_{i+1} -> s_{i+1} (通过 Wps)
    
    Args:
        s_input: 输入的 sensory 向量 (4096维)
        askfor: "cause" 或 "effect"
        Wsp: Place to Sensory 矩阵
        Wps: Sensory to Place 矩阵
        rrm_model: RRM模型实例
        source_id: 源事件ID（用于RRM采样）
        sampling_mode: "direct", "Boltzmann" 或 "max_prob"
        statistic: 是否返回中间向量（用于外部误差计算）
    
    Returns:
        如果 statistic=False:
            (s_output, sampled_slot): 预测向量和采样slot
        如果 statistic=True:
            (s_output, sampled_slot, stats_dict): 预测向量、采样slot和统计字典
            stats_dict 只包含中间步骤的向量（误差在外部计算）
    """
    # 归一化输入
    s = normalize_vec(s_input)
    
    # Step 1: Sensory -> Place cell (使用 Wsp，不是 Wps！)
    p_i = sensory_to_place(s, Wps)
    
    # Step 2: Place cell -> Grid cell (noisy)
    g_i_noisy = place_to_grid_noisy(p_i, Wgp)
    
    # Step 3: Grid cell 清洗 (使用 module_wise_NN_2d)
    gin = g_i_noisy.reshape(1, -1, 1)  # (1, Ng, 1)
    g_i_clean = module_wise_NN_2d(gin, module_gbooks, module_sizes)
    g_i_clean = g_i_clean[0, :, 0]  # (Ng,)
    Ng = g_i_clean.shape[0]
    
    # Step 4: 从RRM采样一个slot
    sampled_slot, probs = rrm_model.sample(source_id, mode=sampling_mode)
    
    # Step 5: Path Integration
    # x方向由askfor决定
    if askfor == "effect":
        Wgg_x = Wgg_c2e  # cause -> effect (x+1)
    else:
        Wgg_x = Wgg_e2c  # effect -> cause (x-1)
    
    # 先做x方向的path integration
    g_next = Wgg_x @ g_i_clean
    
    # y方向由slot决定
    delta_y = get_vertical_shift(sampled_slot)
    if delta_y != 0:
        # y方向的path integration
        direction_y = 1 if delta_y > 0 else -1
        Wgg_y = path_integration_Wgg_2d(lambdas, Ng, axis=1, direction=direction_y)
        for _ in range(abs(delta_y)):
            g_next = Wgg_y @ g_next
    
    # 注意：path_integration后g_next已经是clean的（Wgg是精确置换矩阵），无需再清洗
    
    # Step 6: Grid cell -> Place cell (需要nonlin!)
    p_next = grid_to_place_with_nonlin(g_next, Wpg, thresh=thresh)
    
    # Step 7: Place cell -> Sensory 
    s_output = place_to_sensory(p_next, Wsp)
    
    # 如果需要统计信息（只返回中间向量，误差在外部计算）
    if statistic:
        stats_dict = {
            'p_i': p_i,
            'g_i_noisy': g_i_noisy,
            'g_i_clean': g_i_clean,
            'g_next': g_next,
            'p_next': p_next,
            's_output': s_output,
            'delta_y': delta_y,
        }
        return s_output, sampled_slot, stats_dict
    
    return s_output, sampled_slot

print("✅ Vector-HASH inference functions defined!")


## Section 6: 推理评估 - Gemma centroid vs V-H Reconstructed

**核心对比**：
- 方法1：Gemma 答案 centroid（最大 cluster 的聚类中心向量）
- 方法2：V-H Reconstructed vector（经过 Vector-HASH 重建后的向量）

In [ ]:
# ===== Cell 7: 推理评估 =====
"""
对比两种方法在因果推理任务上的性能

数据结构说明 (来自 causal_data_processing.py):
  - askfor='cause': 输入=Effect (premise), Gemma推断=Cause → causal_A[sample_idx]
  - askfor='effect': 输入=Cause (premise), Gemma推断=Effect → causal_B[sample_idx]
"""

print("="*70)
print("Inference Accuracy Comparison: Gemma Centroid vs V-H")
print("="*70)

# 确保有 H1/H2 embeddings
sample_with_embeddings = [idx for idx in index_to_largest_sample.keys() 
                          if idx in e_care_data and 'h1_vec' in e_care_data[idx]]
print(f"✅ Found {len(sample_with_embeddings)} samples with H1/H2 embeddings")

# 初始化 RRM 模型用于推理
rrm_model_infer = RRM_Model(max_effects=max_effects)

# 开始评估
inference_results = []

for idx in tqdm(sample_with_embeddings, desc="Inference evaluation"):
    # 获取最大 cluster 对应的 sample_idx
    sample_idx = index_to_largest_sample[idx]
    
    # 获取元数据
    meta = causal_metadata[sample_idx]
    askfor = meta['ASKFOR']
    
    # 获取 H1/H2 向量和真实标签
    h1_vec = e_care_data[idx]['h1_vec']
    h2_vec = e_care_data[idx]['h2_vec']
    true_label = e_care_data[idx]['label']
    
    # ===== 方法 1: Gemma 答案 centroid =====
    # askfor='cause' → Gemma推断的Cause centroid 在 causal_A
    # askfor='effect' → Gemma推断的Effect centroid 在 causal_B
    if askfor == 'cause':
        gemma_centroid = causal_A[sample_idx]
        premise_vec = causal_B[sample_idx]
    else:  # askfor == 'effect'
        gemma_centroid = causal_B[sample_idx]
        premise_vec = causal_A[sample_idx]
    
    # 计算 Gemma centroid 与 H1/H2 的相似度
    sim_gemma_h1 = cosine_similarity(gemma_centroid, h1_vec)
    sim_gemma_h2 = cosine_similarity(gemma_centroid, h2_vec)
    pred_gemma = 0 if sim_gemma_h1 >= sim_gemma_h2 else 1
    correct_gemma = (pred_gemma == true_label)
    
    # ===== 方法 2: V-H Reconstructed vector =====
    try:
        # 构造 RRM 的 source_id
        source_id = f"{askfor}_{idx}"
        
        # 调用完整的 Vector-HASH 推理流程
        s_reconstructed, sampled_slot = vector_hash_infer_with_rrm(
            s_input=premise_vec,
            askfor=askfor,
            Wps=Wps,
            Wsp=Wsp,
            Wgp=Wgp,
            Wpg=Wpg,
            Wgg_c2e=Wgg_c2e,
            Wgg_e2c=Wgg_e2c,
            module_gbooks=module_gbooks,
            module_sizes=module_sizes,
            rrm_model=rrm_model_infer,
            source_id=source_id,
            sampling_mode="max_prob"
        )
    except Exception as e:
        # 如果推理失败，使用 premise_vec 作为降级方案
        s_reconstructed = premise_vec
        sampled_slot = -1
        # print(f"  Warning: inference failed for {idx}: {e}")
    
    # 计算重建向量与 H1/H2 的相似度
    sim_recon_h1 = cosine_similarity(s_reconstructed, h1_vec)
    sim_recon_h2 = cosine_similarity(s_reconstructed, h2_vec)
    pred_recon = 0 if sim_recon_h1 >= sim_recon_h2 else 1
    correct_recon = (pred_recon == true_label)
    
    # 计算重建向量与目标向量 (Gemma centroid) 的相似度（用于重建率分析）
    sim_recon_target = cosine_similarity(s_reconstructed, gemma_centroid)
    
    # 记录结果
    inference_results.append({
        'INDEX': idx,
        'sample_idx': sample_idx,
        'askfor': askfor,
        'true_label': true_label,
        'sim_gemma_h1': float(sim_gemma_h1),
        'sim_gemma_h2': float(sim_gemma_h2),
        'pred_gemma': int(pred_gemma),
        'correct_gemma': bool(correct_gemma),
        'sim_recon_h1': float(sim_recon_h1),
        'sim_recon_h2': float(sim_recon_h2),
        'pred_recon': int(pred_recon),
        'correct_recon': bool(correct_recon),
        'sampled_slot': int(sampled_slot),
        'sim_recon_target': float(sim_recon_target)  # V-H重建向量与Gemma centroid的相似度
    })

print(f"\n✅ Evaluated {len(inference_results)} samples")

# ===== 统计结果 =====
correct_gemma_arr = np.array([r['correct_gemma'] for r in inference_results])
correct_recon_arr = np.array([r['correct_recon'] for r in inference_results])

print("\n" + "="*70)
print("Inference Accuracy Comparison")
print("="*70)
print(f"{'Method':<50} {'Accuracy':>15} {'Count':>15}")
print("-"*70)
print(f"{'Gemma Centroid (Largest Cluster)':<50} {correct_gemma_arr.mean():>15.2%} {f'{correct_gemma_arr.sum()}/{len(correct_gemma_arr)}':>15}")
print(f"{'V-H Reconstructed Vector':<50} {correct_recon_arr.mean():>15.2%} {f'{correct_recon_arr.sum()}/{len(correct_recon_arr)}':>15}")
print("-"*70)
print(f"{'Random Baseline':<50} {'50.00%':>15}")

# 按 askfor 分量分析
print("\n" + "="*70)
print("Performance by askfor type")
print("="*70)
for askfor_type in ['cause', 'effect']:
    subset = [r for r in inference_results if r['askfor'] == askfor_type]
    if subset:
        subset_arr = np.array([r['correct_recon'] for r in subset])
        gemma_subset_arr = np.array([r['correct_gemma'] for r in subset])
        print(f"\n{askfor_type.upper()}:")
        print(f"  Gemma:  {gemma_subset_arr.mean():.2%} ({gemma_subset_arr.sum()}/{len(gemma_subset_arr)})")
        print(f"  V-H:    {subset_arr.mean():.2%} ({subset_arr.sum()}/{len(subset_arr)})")


## Section 7: 重建率分析

比较 $\hat{s}_{i+1}$ 和最大 Cluster 对应的 Centroid 向量的相似度

In [ ]:
# ===== Cell 8: 重建率分析 =====
"""
分析 Vector-HASH 重建的准确性

对每个样本计算：
1. 输入: premise_vec
2. 预期输出: Gemma centroid (最大 cluster)
3. 实际输出: V-H reconstructed vector (s_reconstructed)
4. 相似度分析: cos_sim(s_reconstructed, gemma_centroid)
5. 答案变化分析: 重建不完美导致的答案改变

推理流程：
s_i → p_i → g_i^{noisy} → g_i^{clean} → g_{i+1}^{clean} → p_{i+1} → s_{i+1}
"""

print("="*70)
print("Vector-HASH Reconstruction Analysis")
print("="*70)

reconstruction_analysis = []

# 用于统计答案变化的计数器
answer_change_stats = {
    'originally_correct_now_wrong': 0,  # 原本对，但因重建不完美选错了
    'originally_wrong_now_correct': 0,  # 原本错，但因重建不完美反而选对了
    'both_correct': 0,                  # 两者都对
    'both_wrong': 0,                    # 两者都错
}

answer_change_details = {
    'originally_correct_now_wrong': [],
    'originally_wrong_now_correct': [],
}

for r in tqdm(inference_results, desc="Reconstruction analysis"):
    idx = r['INDEX']
    sample_idx = r['sample_idx']
    askfor = r['askfor']
    
    # 直接使用已计算的 s_reconstructed 与 target (Gemma centroid) 的相似度
    # sim_recon_target 代表 V-H 重建向量与目标向量的相似度
    # 如果相似度 = 1，说明重建完美；相似度越低，重建质量越差
    reconstruction_similarity = r['sim_recon_target']
    
    # 对标：Gemma centroid 自己与选中的假设（H1或H2）的相似度
    # 这代表 Gemma 答案的"置信度"
    gemma_confidence = (r['sim_gemma_h1'] if r['pred_gemma'] == 0 else r['sim_gemma_h2'])
    
    # 分析答案变化情况
    pred_gemma = r['pred_gemma']
    pred_recon = r['pred_recon']
    true_label = r['true_label']
    correct_gemma = r['correct_gemma']
    correct_recon = r['correct_recon']
    
    # 判断答案是否因重建不完美而改变
    answer_changed = (pred_gemma != pred_recon)
    
    if answer_changed:
        if correct_gemma and not correct_recon:
            # 原本对（Gemma centroid选对了），但因重建不完美而选错了
            answer_change_stats['originally_correct_now_wrong'] += 1
            answer_change_details['originally_correct_now_wrong'].append({
                'INDEX': idx,
                'askfor': askfor,
                'reconstruction_similarity': reconstruction_similarity,
                'gemma_confidence': gemma_confidence,
                'pred_gemma': pred_gemma,
                'pred_recon': pred_recon,
                'true_label': true_label
            })
        elif not correct_gemma and correct_recon:
            # 原本错（Gemma centroid选错了），但因重建不完美反而选对了
            answer_change_stats['originally_wrong_now_correct'] += 1
            answer_change_details['originally_wrong_now_correct'].append({
                'INDEX': idx,
                'askfor': askfor,
                'reconstruction_similarity': reconstruction_similarity,
                'gemma_confidence': gemma_confidence,
                'pred_gemma': pred_gemma,
                'pred_recon': pred_recon,
                'true_label': true_label
            })
    else:
        # 答案没变
        if correct_gemma:
            answer_change_stats['both_correct'] += 1
        else:
            answer_change_stats['both_wrong'] += 1
    
    reconstruction_analysis.append({
        'INDEX': idx,
        'askfor': askfor,
        'reconstruction_similarity': float(reconstruction_similarity),
        'gemma_confidence': float(gemma_confidence),
        'similarity_delta': float(reconstruction_similarity - gemma_confidence),
        'vh_correct': r['correct_recon'],
        'gemma_correct': r['correct_gemma'],
        'answer_changed': answer_changed
    })

print(f"\n✅ Analyzed reconstruction for {len(reconstruction_analysis)} samples")

# 统计
recon_sim_arr = np.array([r['reconstruction_similarity'] for r in reconstruction_analysis])
gemma_conf_arr = np.array([r['gemma_confidence'] for r in reconstruction_analysis])
sim_delta_arr = np.array([r['similarity_delta'] for r in reconstruction_analysis])

print("\n" + "="*70)
print("Reconstruction Quality Analysis")
print("="*70)
print("\n说明:")
print("  - 重建相似度: V-H重建向量(s_reconstructed) 与 Gemma centroid 的余弦相似度")
print("  - Gemma置信度: Gemma centroid 与其选中假设(H1/H2)的余弦相似度")
print("  - 如果重建相似度 = 1.0，说明V-H完美重建了Gemma centroid")
print("="*70)

print(f"\nV-H 重建相似度 (与 Gemma centroid):")
print(f"  Mean:     {recon_sim_arr.mean():.4f}")
print(f"  Std dev:  {recon_sim_arr.std():.4f}")
print(f"  Min:      {recon_sim_arr.min():.4f}")
print(f"  Max:      {recon_sim_arr.max():.4f}")
print(f"  Median:   {np.median(recon_sim_arr):.4f}")

print(f"\nGemma Centroid 置信度 (与其选中的假设):")
print(f"  Mean:     {gemma_conf_arr.mean():.4f}")
print(f"  Std dev:  {gemma_conf_arr.std():.4f}")

print(f"\n相似度差异 (V-H重建相似度 - Gemma置信度):")
print(f"  Mean delta:      {sim_delta_arr.mean():.4f}")
print(f"  Positive delta:  {(sim_delta_arr > 0).sum()} samples ({(sim_delta_arr > 0).mean():.2%})")
print(f"  Negative delta:  {(sim_delta_arr < 0).sum()} samples ({(sim_delta_arr < 0).mean():.2%})")

# 按百分位数统计
print(f"\nPercentile Analysis (V-H 重建相似度):")
for p in [25, 50, 75, 90, 95]:
    print(f"  {p}th percentile: {np.percentile(recon_sim_arr, p):.4f}")

# ===== 答案变化分析 =====
print("\n" + "="*70)
print("Answer Change Analysis (由于重建不完美导致的答案变化)")
print("="*70)
print("\n说明:")
print("  - '原本' 指使用 Gemma centroid 的预测结果")
print("  - '重建后' 指使用 V-H reconstructed vector 的预测结果")
print("="*70)

total_samples = len(reconstruction_analysis)
print(f"\n总样本数: {total_samples}")
print(f"\n答案一致 (pred_gemma == pred_recon):")
print(f"  两者都对: {answer_change_stats['both_correct']} ({answer_change_stats['both_correct']/total_samples:.2%})")
print(f"  两者都错: {answer_change_stats['both_wrong']} ({answer_change_stats['both_wrong']/total_samples:.2%})")

print(f"\n答案改变 (pred_gemma != pred_recon):")
print(f"  原本对 → 重建后错: {answer_change_stats['originally_correct_now_wrong']} ({answer_change_stats['originally_correct_now_wrong']/total_samples:.2%})")
print(f"  原本错 → 重建后对: {answer_change_stats['originally_wrong_now_correct']} ({answer_change_stats['originally_wrong_now_correct']/total_samples:.2%})")

total_changed = answer_change_stats['originally_correct_now_wrong'] + answer_change_stats['originally_wrong_now_correct']
print(f"\n总答案改变数: {total_changed} ({total_changed/total_samples:.2%})")

if answer_change_stats['originally_correct_now_wrong'] > 0:
    # 分析"原本对但选错"的样本的重建相似度
    wrong_samples = answer_change_details['originally_correct_now_wrong']
    wrong_sim = [s['reconstruction_similarity'] for s in wrong_samples]
    print(f"\n'原本对→重建后错' 样本的重建相似度分析:")
    print(f"  Mean: {np.mean(wrong_sim):.4f}")
    print(f"  Median: {np.median(wrong_sim):.4f}")

if answer_change_stats['originally_wrong_now_correct'] > 0:
    # 分析"原本错但选对"的样本的重建相似度
    correct_samples = answer_change_details['originally_wrong_now_correct']
    correct_sim = [s['reconstruction_similarity'] for s in correct_samples]
    print(f"\n'原本错→重建后对' 样本的重建相似度分析:")
    print(f"  Mean: {np.mean(correct_sim):.4f}")
    print(f"  Median: {np.median(correct_sim):.4f}")

# 计算净影响（负面影响 - 正面影响）
net_negative_impact = answer_change_stats['originally_correct_now_wrong'] - answer_change_stats['originally_wrong_now_correct']
print(f"\n净负面影响 (原本对→错 - 原本错→对): {net_negative_impact}")
if net_negative_impact > 0:
    print(f"  → 重建不完美导致净损失 {net_negative_impact} 个正确答案")
elif net_negative_impact < 0:
    print(f"  → 重建不完美反而带来净增加 {-net_negative_impact} 个正确答案")
else:
    print(f"  → 重建不完美的正负影响相互抵消")


## Section 7.5: 逐步重建误差分析

分析 Vector-HASH 推理流程中每一步的重建误差：
$$s_i \rightarrow p_i \rightarrow g_i^{noisy} \rightarrow g_i^{clean} \Rightarrow g_{i+1}^{clean} \rightarrow p_{i+1} \rightarrow s_{i+1}$$

对于每一步，计算实际重建向量与真实目标向量的误差。

In [ ]:
# ===== Cell 7.5: 逐步重建误差分析（使用 statistic=True） =====
"""
分析推理流程每一步的重建误差

推理流程：
1. s_i -> p_i (通过 Wps)
2. p_i -> g_i^noisy (通过 Wgp)
3. g_i^noisy -> g_i^clean (通过 module_wise_NN_2d)
4. 从RRM采样slot，决定y方向偏移
5. Path Integration: g_i^clean -> g_{i+1}^clean (通过 Wgg，包含x和y方向)
6. g_{i+1}^clean -> p_{i+1} (通过 Wpg + nonlin)
7. p_{i+1} -> s_{i+1} (通过 Wsp)

误差计算：完全在外部计算，与真实目标向量（从pbook/gbook获取）对比
注意：由于RRM采样决定y方向偏移，真实的目标位置需要根据采样结果确定
"""

print("="*70)
print("Step-by-Step Reconstruction Error Analysis (RRM-aware)")
print("="*70)

# 加载位置信息
positions_path = os.path.join(RESULTS_DIR, 'positions.json')
if not os.path.exists(positions_path):
    print(f"⚠️ {positions_path} not found. Skipping step-by-step error analysis.")
else:
    with open(positions_path, 'r', encoding='utf-8') as f:
        positions_data = json.load(f)
    
    premise_locs = positions_data.get('premise_locs', {})
    print(f"✅ Loaded {len(premise_locs)} premise locations")
    
    # 用于存储每一步的误差
    step_errors = {
        'err_p_i': [],       # p_i vs true_p_i
        'err_g_i_noisy': [], # g_i^noisy vs true_g_i
        'err_g_i_clean': [], # g_i^clean vs true_g_i  
        'err_g_next': [],    # g_{i+1}^clean vs true_g_{i+1} (RRM-aware)
        'err_p_next': [],    # p_{i+1} vs true_p_{i+1} (RRM-aware)
        'err_s_next': [],    # s_{i+1} vs true_s_{i+1} (gemma_centroid)
    }
    
    step_error_details = []
    skipped_count = 0
    error_count = 0
    
    for r in tqdm(inference_results[:100], desc="Computing step-by-step errors"):  # 先处理前100个样本作为示例
        idx = r['INDEX']
        sample_idx = r['sample_idx']
        askfor = r['askfor']
        
        # 构造位置key
        loc_key = f"{idx}_{askfor}"
        
        if loc_key not in premise_locs:
            skipped_count += 1
            continue
        
        # 获取premise位置
        pos_i = premise_locs[loc_key]
        x_i, y_i = pos_i[0], pos_i[1]
        
        # 检查当前位置是否在合法范围内
        if not (0 <= x_i < Npos and 0 <= y_i < Npos):
            skipped_count += 1
            continue
        
        # 获取输入向量和真实目标向量
        if askfor == 'cause':
            s_i = causal_B[sample_idx]  # premise (Effect)
            gemma_centroid = causal_A[sample_idx]  # target (Cause)
        else:
            s_i = causal_A[sample_idx]  # premise (Cause)
            gemma_centroid = causal_B[sample_idx]  # target (Effect)
        
        # 获取当前位置的真实向量
        true_p_i = pbook[:, x_i, y_i]
        true_g_i = gbook[:, x_i, y_i]
        
        try:
            # 调用推理函数（statistic=True，获取中间向量）
            result = vector_hash_infer_with_rrm(
                s_input=s_i,
                askfor=askfor,
                Wps=Wps, Wsp=Wsp, Wgp=Wgp, Wpg=Wpg,
                Wgg_c2e=Wgg_c2e, Wgg_e2c=Wgg_e2c,
                module_gbooks=module_gbooks,
                module_sizes=module_sizes,
                rrm_model=rrm_model_infer,
                source_id=idx,
                sampling_mode="max_prob",
                statistic=True
            )
            
            # 检查返回值
            if len(result) != 3:
                print(f"⚠️ Sample {idx}: unexpected return length {len(result)}")
                error_count += 1
                continue
                
            s_output, sampled_slot, stats_dict = result
            
        except Exception as e:
            if error_count < 5:  # 只打印前5个错误
                print(f"⚠️ Error on sample {idx} (askfor={askfor}): {e}")
            error_count += 1
            continue
        
        # 从stats_dict获取delta_y和中间向量
        delta_y = stats_dict['delta_y']
        p_i = stats_dict['p_i']
        g_i_noisy = stats_dict['g_i_noisy']
        g_i_clean = stats_dict['g_i_clean']
        g_next = stats_dict['g_next']
        p_next = stats_dict['p_next']
        
        # 计算实际的目标位置（考虑RRM采样）
        if askfor == 'effect':
            x_next = x_i + 1
        else:
            x_next = x_i - 1
        y_next = y_i + delta_y
        
        # 检查目标位置是否在合法范围内
        if not (0 <= x_next < Npos and 0 <= y_next < Npos):
            skipped_count += 1
            continue
        
        # 获取真实的目标向量（RRM-aware）
        true_p_next = pbook[:, x_next, y_next]
        true_g_next = gbook[:, x_next, y_next]
        
        # ===== 在外部计算所有误差 =====
        err_p_i = np.sum(np.abs(p_i - true_p_i)) / (2 * Np)
        err_g_i_noisy = np.sum(np.abs(g_i_noisy - true_g_i)) / (2 * Ng)
        err_g_i_clean = np.sum(np.abs(g_i_clean - true_g_i)) / (2 * Ng)
        err_g_next = np.sum(np.abs(g_next - true_g_next)) / (2 * Ng)
        err_p_next = np.sum(np.abs(p_next - true_p_next)) / (2 * Np)
        err_s_next = np.sum(np.abs(s_output - gemma_centroid)) / (2 * Ns)
        
        # 记录误差
        step_errors['err_p_i'].append(err_p_i)
        step_errors['err_g_i_noisy'].append(err_g_i_noisy)
        step_errors['err_g_i_clean'].append(err_g_i_clean)
        step_errors['err_g_next'].append(err_g_next)
        step_errors['err_p_next'].append(err_p_next)
        step_errors['err_s_next'].append(err_s_next)
        
        step_error_details.append({
            'INDEX': idx,
            'askfor': askfor,
            'pos_i': [int(x_i), int(y_i)],
            'pos_next': [int(x_next), int(y_next)],
            'delta_y': int(delta_y),
            'sampled_slot': int(sampled_slot),
            'err_p_i': float(err_p_i),
            'err_g_i_noisy': float(err_g_i_noisy),
            'err_g_i_clean': float(err_g_i_clean),
            'err_g_next': float(err_g_next),
            'err_p_next': float(err_p_next),
            'err_s_next': float(err_s_next)
        })
    
    print(f"\n✅ Computed step-by-step errors for {len(step_error_details)} samples")
    print(f"⚠️ Skipped {skipped_count} samples (missing position or out of range)")
    print(f"❌ Error count: {error_count} samples")
    
    # 统计结果
    if len(step_error_details) > 0:
        print("\n" + "="*70)
        print("Step-by-Step Reconstruction Error Statistics (RRM-aware)")
        print("="*70)
        print("\n误差定义: err = sum(|predicted - true|) / (2 * N)")
        print("  其中 N 是该层的维度 (Np, Ng, 或 Ns)")
        print("  误差范围: [0, 1], 0表示完美重建, 1表示完全相反")
        print("="*70)
        
        step_names = [
            ('err_p_i',       's_i → p_i'),
            ('err_g_i_noisy', 'p_i → g_i^noisy'),
            ('err_g_i_clean', 'g_i^noisy → g_i^clean (cleanup)'),
            ('err_g_next',    'g_i^clean → g_{i+1}^clean (path integration + RRM)'),
            ('err_p_next',    'g_{i+1}^clean → p_{i+1}'),
            ('err_s_next',    'p_{i+1} → s_{i+1} (vs gemma_centroid)'),
        ]
        
        print(f"\n{'Step':<55} {'Mean Err':>12} {'Std':>12} {'Min':>12} {'Max':>12}")
        print("-"*100)
        for key, name in step_names:
            errs = np.array(step_errors[key])
            print(f"{name:<55} {errs.mean():>12.4f} {errs.std():>12.4f} {errs.min():>12.4f} {errs.max():>12.4f}")
        
        # 累积误差分析
        print("\n" + "="*70)
        print("Cumulative Error Analysis (误差如何在流程中累积)")
        print("="*70)
        total_err = (np.array(step_errors['err_p_i']) + 
                     np.array(step_errors['err_g_i_noisy']) +
                     np.array(step_errors['err_g_i_clean']) +
                     np.array(step_errors['err_g_next']) +
                     np.array(step_errors['err_p_next']) +
                     np.array(step_errors['err_s_next']))
        print(f"Total accumulated error (mean): {total_err.mean():.4f}")
        print(f"Total accumulated error (std):  {total_err.std():.4f}")
        
        # 找出误差最大的步骤
        mean_errors = {name: np.mean(step_errors[key]) for key, name in step_names}
        worst_step = max(mean_errors.items(), key=lambda x: x[1])
        best_step = min(mean_errors.items(), key=lambda x: x[1])
        print(f"\n误差最大的步骤: {worst_step[0]} (mean error = {worst_step[1]:.4f})")
        print(f"误差最小的步骤: {best_step[0]} (mean error = {best_step[1]:.4f})")
        
        # RRM采样统计
        delta_y_values = [d['delta_y'] for d in step_error_details]
        print("\n" + "="*70)
        print("RRM Sampling Statistics (y方向偏移统计)")
        print("="*70)
        delta_y_counts = {}
        for dy in delta_y_values:
            delta_y_counts[dy] = delta_y_counts.get(dy, 0) + 1
        for dy in sorted(delta_y_counts.keys()):
            print(f"  delta_y = {dy:+2d}: {delta_y_counts[dy]:4d} samples ({100*delta_y_counts[dy]/len(delta_y_values):.1f}%)")
        
        # 保存详细结果
        step_error_path = os.path.join(RESULTS_DIR, 'step_by_step_errors_rrm_aware.json')
        with open(step_error_path, 'w', encoding='utf-8') as f:
            json.dump({
                'statistics': {key: {
                    'mean': float(np.mean(step_errors[key])),
                    'std': float(np.std(step_errors[key])),
                    'min': float(np.min(step_errors[key])),
                    'max': float(np.max(step_errors[key]))
                } for key, _ in step_names},
                'rrm_sampling': {
                    'delta_y_distribution': delta_y_counts,
                },
                'detailed': step_error_details
            }, f, ensure_ascii=False, indent=2)
        print(f"\n✅ Step-by-step errors (RRM-aware) saved to: {step_error_path}")


## Section 8: 结果保存和可视化

In [ ]:
# ===== Cell 9: 保存评估结果 =====
import json
from datetime import datetime

print("="*60)
print("Saving Evaluation Results")
print("="*60)

# 准备保存的数据
results_summary = {
    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'total_samples': len(inference_results),
    'model_config': {
        'lambdas': lambdas,
        'Ng': int(Ng),
        'Npos': int(Npos),
        'Np': int(Np),
        'Ns': int(Ns),
        'max_effects': max_effects,
        'thresh': float(thresh)
    },
    'gemma_centroid': {
        'accuracy': float(correct_gemma_arr.mean()),
        'correct_count': int(correct_gemma_arr.sum()),
        'total_count': len(correct_gemma_arr)
    },
    'vh_reconstructed': {
        'accuracy': float(correct_recon_arr.mean()),
        'correct_count': int(correct_recon_arr.sum()),
        'total_count': len(correct_recon_arr)
    },
    'reconstruction_analysis': {
        'mean_reconstruction_similarity': float(recon_sim_arr.mean()),
        'std_reconstruction_similarity': float(recon_sim_arr.std()),
        'min_reconstruction_similarity': float(recon_sim_arr.min()),
        'max_reconstruction_similarity': float(recon_sim_arr.max()),
        'mean_gemma_confidence': float(gemma_conf_arr.mean()),
        'mean_similarity_delta': float(sim_delta_arr.mean()),
        'positive_delta_samples': int((sim_delta_arr > 0).sum()),
        'negative_delta_samples': int((sim_delta_arr < 0).sum())
    },
    'answer_change_analysis': {
        'total_samples': len(reconstruction_analysis),
        'both_correct': answer_change_stats['both_correct'],
        'both_wrong': answer_change_stats['both_wrong'],
        'originally_correct_now_wrong': answer_change_stats['originally_correct_now_wrong'],
        'originally_wrong_now_correct': answer_change_stats['originally_wrong_now_correct'],
        'total_answer_changed': answer_change_stats['originally_correct_now_wrong'] + answer_change_stats['originally_wrong_now_correct'],
        'net_negative_impact': answer_change_stats['originally_correct_now_wrong'] - answer_change_stats['originally_wrong_now_correct']
    }
}

# 按 askfor 分组统计
for askfor_type in ['cause', 'effect']:
    subset = [r for r in inference_results if r['askfor'] == askfor_type]
    if subset:
        results_summary[f'{askfor_type}_analysis'] = {
            'count': len(subset),
            'gemma_accuracy': float(np.mean([r['correct_gemma'] for r in subset])),
            'vh_accuracy': float(np.mean([r['correct_recon'] for r in subset]))
        }

# 保存摘要
summary_path = os.path.join(RESULTS_DIR, 'inference_evaluation.json')
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(results_summary, f, ensure_ascii=False, indent=2)
print(f"✅ Summary saved to: {summary_path}")

# 保存详细结果
detailed_path = os.path.join(RESULTS_DIR, 'inference_detailed.json')
with open(detailed_path, 'w', encoding='utf-8') as f:
    json.dump(inference_results, f, ensure_ascii=False, indent=2)
print(f"✅ Detailed results saved to: {detailed_path}")

# 保存重建分析结果（包含答案变化详情）
reconstruction_path = os.path.join(RESULTS_DIR, 'reconstruction_analysis.json')
with open(reconstruction_path, 'w', encoding='utf-8') as f:
    json.dump({
        'timestamp': results_summary['timestamp'],
        'statistics': results_summary['reconstruction_analysis'],
        'answer_change_statistics': results_summary['answer_change_analysis'],
        'answer_change_details': answer_change_details,
        'detailed': reconstruction_analysis
    }, f, ensure_ascii=False, indent=2)
print(f"✅ Reconstruction analysis saved to: {reconstruction_path}")

print("\n" + "="*60)
print("EVALUATION SUMMARY")
print("="*60)
print(f"""Total samples: {results_summary['total_samples']}

Model Configuration:
  lambdas: {results_summary['model_config']['lambdas']}
  Ng: {results_summary['model_config']['Ng']}
  Npos: {results_summary['model_config']['Npos']}
  Np: {results_summary['model_config']['Np']}

Inference Accuracy:
  Gemma Centroid:           {results_summary['gemma_centroid']['accuracy']:.2%} ({results_summary['gemma_centroid']['correct_count']}/{results_summary['gemma_centroid']['total_count']})
  V-H Reconstructed:       {results_summary['vh_reconstructed']['accuracy']:.2%} ({results_summary['vh_reconstructed']['correct_count']}/{results_summary['vh_reconstructed']['total_count']})
  Performance delta:       {results_summary['vh_reconstructed']['accuracy'] - results_summary['gemma_centroid']['accuracy']:+.2%}

Reconstruction Quality:
  Mean V-H→Gemma similarity:      {results_summary['reconstruction_analysis']['mean_reconstruction_similarity']:.4f}
  Std deviation:                  {results_summary['reconstruction_analysis']['std_reconstruction_similarity']:.4f}
  Mean Gemma confidence (→H1/H2): {results_summary['reconstruction_analysis']['mean_gemma_confidence']:.4f}
  Mean similarity delta (V-H-Gemma): {results_summary['reconstruction_analysis']['mean_similarity_delta']:+.4f}

Answer Change Impact (重建不完美的影响):
  原本对→重建后错: {results_summary['answer_change_analysis']['originally_correct_now_wrong']} ({results_summary['answer_change_analysis']['originally_correct_now_wrong']/results_summary['total_samples']:.2%})
  原本错→重建后对: {results_summary['answer_change_analysis']['originally_wrong_now_correct']} ({results_summary['answer_change_analysis']['originally_wrong_now_correct']/results_summary['total_samples']:.2%})
  净负面影响:      {results_summary['answer_change_analysis']['net_negative_impact']}

Random Baseline: 50.00%
""")


In [ ]:
# ===== Cell 10: Visualization =====
"""
Visualization of inference results and step-by-step reconstruction errors
"""

import matplotlib.pyplot as plt
import matplotlib as mpl

# Set global font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['legend.fontsize'] = 10

# Define color palette (sophisticated colors)
colors_palette = {
    'gemma_cause': '#2E86AB',      # Steel Blue
    'gemma_effect': '#A23B72',     # Dark Pink
    'vh_cause': '#F18F01',         # Orange
    'vh_effect': '#C73E1D',        # Red
    'correct_to_wrong': '#E63946', # Bright Red
    'wrong_to_correct': '#2A9D8F', # Teal
    'random': '#6C757D',           # Gray
}

# ===== Figure 1: Inference Accuracy & Answer Change Impact =====
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('Vector-HASH Inference Evaluation', fontsize=16, fontweight='bold', y=1.02)

# --- Left subplot: Inference Accuracy by askfor type ---
ax1 = axes[0]

# Calculate accuracy by askfor
cause_results = [r for r in inference_results if r['askfor'] == 'cause']
effect_results = [r for r in inference_results if r['askfor'] == 'effect']

gemma_cause_acc = np.mean([r['correct_gemma'] for r in cause_results]) * 100
gemma_effect_acc = np.mean([r['correct_gemma'] for r in effect_results]) * 100
vh_cause_acc = np.mean([r['correct_recon'] for r in cause_results]) * 100
vh_effect_acc = np.mean([r['correct_recon'] for r in effect_results]) * 100

# Bar positions
x = np.arange(2)  # Cause, Effect
width = 0.35

bars1 = ax1.bar(x - width/2, [gemma_cause_acc, gemma_effect_acc], width, 
                label='Gemma Centroid', color=[colors_palette['gemma_cause'], colors_palette['gemma_effect']], 
                edgecolor='white', linewidth=1.5, alpha=0.9)
bars2 = ax1.bar(x + width/2, [vh_cause_acc, vh_effect_acc], width, 
                label='V-H Reconstructed', color=[colors_palette['vh_cause'], colors_palette['vh_effect']], 
                edgecolor='white', linewidth=1.5, alpha=0.9)

# Add value labels on bars
def add_value_labels(ax, bars, fmt='{:.1f}%'):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(fmt.format(height),
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=11, fontweight='bold')

add_value_labels(ax1, bars1)
add_value_labels(ax1, bars2)

# Add random baseline
ax1.axhline(y=50, color=colors_palette['random'], linestyle='--', linewidth=2, alpha=0.7, label='Random Baseline')

ax1.set_ylabel('Accuracy (%)', fontsize=12)
ax1.set_xlabel('ASKFOR Type', fontsize=12)
ax1.set_title('Inference Accuracy by ASKFOR Type', fontsize=13, fontweight='bold', pad=10)
ax1.set_xticks(x)
ax1.set_xticklabels(['Cause', 'Effect'])
ax1.set_ylim(0, 100)
ax1.legend(loc='upper right', framealpha=0.9)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# --- Right subplot: Answer Change Impact ---
ax2 = axes[1]

# Data for answer change
categories = ['Correct→Wrong\n(Negative)', 'Wrong→Correct\n(Positive)']
values = [
    answer_change_stats['originally_correct_now_wrong'],
    answer_change_stats['originally_wrong_now_correct']
]
percentages = [v / len(reconstruction_analysis) * 100 for v in values]

bar_colors = [colors_palette['correct_to_wrong'], colors_palette['wrong_to_correct']]
bars3 = ax2.bar(categories, values, color=bar_colors, edgecolor='white', linewidth=1.5, alpha=0.9)

# Add value labels
for bar, val, pct in zip(bars3, values, percentages):
    height = bar.get_height()
    ax2.annotate(f'{val}\n({pct:.1f}%)',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom', fontsize=11, fontweight='bold')

# Add net impact annotation
net_impact = values[0] - values[1]
impact_text = f'Net Negative Impact: {net_impact}'
ax2.annotate(impact_text, xy=(0.5, 0.95), xycoords='axes fraction',
             ha='center', va='top', fontsize=12, fontweight='bold',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', edgecolor='gray', alpha=0.8))

ax2.set_ylabel('Number of Samples', fontsize=12)
ax2.set_title('Answer Change Impact\n(Due to Imperfect Reconstruction)', fontsize=13, fontweight='bold', pad=10)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print(f"✅ Figure 1 saved to {RESULTS_DIR}")

In [ ]:
# ===== Cell 11: Step-by-Step Reconstruction Error Visualization =====
"""
Visualization of step-by-step reconstruction errors with LaTeX labels
"""

# Set up the figure with sophisticated style
fig, ax = plt.subplots(figsize=(12, 6))

# Define step labels with LaTeX formatting
step_labels = [
    r'$s_i \rightarrow p_i$',
    r'$p_i \rightarrow g_i^{\mathrm{noisy}}$',
    r'$g_i^{\mathrm{noisy}} \rightarrow g_i^{\mathrm{clean}}$',
    r'$g_i^{\mathrm{clean}} \rightarrow g_{i+1}^{\mathrm{clean}}$',
    r'$g_{i+1}^{\mathrm{clean}} \rightarrow p_{i+1}$',
    r'$p_{i+1} \rightarrow s_{i+1}$'
]

# Mean errors for each step
mean_errors = [
    np.mean(step_errors['err_p_i']),
    np.mean(step_errors['err_g_i_noisy']),
    np.mean(step_errors['err_g_i_clean']),
    np.mean(step_errors['err_g_next']),
    np.mean(step_errors['err_p_next']),
    np.mean(step_errors['err_s_next'])
]

# Color gradient from cool to warm
colors_gradient = ['#264653', '#2A9D8F', '#E9C46A', '#F4A261', '#E76F51', '#9B2226']

# Create bar plot
x_pos = np.arange(len(step_labels))
bars = ax.bar(x_pos, mean_errors, color=colors_gradient, edgecolor='white', linewidth=2, alpha=0.9)

# Add value labels on top of bars
for bar, val in zip(bars, mean_errors):
    height = bar.get_height()
    if height > 0.001:  # Only show label if error > 0.001
        ax.annotate(f'{val:.4f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=11, fontweight='bold')
    else:
        ax.annotate(f'{val:.4f}',
                    xy=(bar.get_x() + bar.get_width() / 2, 0.002),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10, color='gray')

# Styling
ax.set_xlabel('Inference Step', fontsize=13, fontweight='bold')
ax.set_ylabel('Mean Reconstruction Error', fontsize=13, fontweight='bold')
ax.set_title('Step-by-Step Reconstruction Error in Vector-HASH Inference', fontsize=15, fontweight='bold', pad=15)
ax.set_xticks(x_pos)
ax.set_xticklabels(step_labels, fontsize=12)
ax.set_ylim(0, max(mean_errors) * 1.25)

# Add grid
ax.grid(axis='y', alpha=0.3, linestyle='--', zorder=0)
ax.set_axisbelow(True)

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add a subtle annotation about error definition
ax.annotate(r'Error: $\frac{\sum |predicted - true|}{2N}$', 
            xy=(0.98, 0.95), xycoords='axes fraction',
            ha='right', va='top', fontsize=11,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='whitesmoke', edgecolor='gray', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"✅ Figure 2 saved to {RESULTS_DIR}")

# Print summary statistics
print("\n" + "="*60)
print("Step-by-Step Error Summary")
print("="*60)
for label, err in zip(step_labels, mean_errors):
    print(f"  {label}: {err:.4f}")

## Section 9: 自重建流程统计测试

本节进行自重建测试，分析训练次数与重建率的关系：
$$s_i \rightarrow p_i \rightarrow g_i^{noisy} \rightarrow g_i^{clean} \rightarrow p_{i} \rightarrow s_{i}$$

**动机**：在训练过程中，premise训练了100次，但每个回答（Cluster的Centroid向量）训练的次数取决于该cluster的样本数量（少于100次）。我们想验证重建率是否与训练次数相关。

**分析内容**：
1. Premise 各步重建率（训练100次）
2. 最大 Cluster 的 Centroid 各步重建率（训练次数=cluster样本数）
3. 重建率与训练次数的关系曲线

In [ ]:
# ===== Cell 9.1: 自重建流程分析 =====
"""
自重建流程：s_i -> p_i -> g_i^noisy -> g_i^clean -> p_i -> s_i
不进行路径积分，测试从输入重建回输入的能力

分析内容：
1. Premise 的自重建（每个premise训练100次）
2. 最大 Cluster Centroid 的自重建（训练次数=cluster样本数）
"""

print("="*70)
print("Self-Reconstruction Analysis (无路径积分)")
print("="*70)

def self_reconstruction(s_input, Wps, Wsp, Wgp, Wpg, module_gbooks, module_sizes, thresh=2.0):
    """
    自重建流程: s_i -> p_i -> g_i^noisy -> g_i^clean -> p_i' -> s_i'
    不进行路径积分
    
    Returns:
        s_output: 重建后的sensory向量
        stats_dict: 包含中间步骤向量的字典
    """
    # 归一化输入
    s = normalize_vec(s_input)
    
    # Step 1: Sensory -> Place cell
    p_i = sensory_to_place(s, Wps)
    
    # Step 2: Place cell -> Grid cell (noisy)
    g_i_noisy = place_to_grid_noisy(p_i, Wgp)
    
    # Step 3: Grid cell 清洗 (使用 module_wise_NN_2d)
    gin = g_i_noisy.reshape(1, -1, 1)  # (1, Ng, 1)
    g_i_clean = module_wise_NN_2d(gin, module_gbooks, module_sizes)
    g_i_clean = g_i_clean[0, :, 0]  # (Ng,)
    
    # Step 4: Grid cell -> Place cell (直接从g_i_clean，不进行路径积分)
    p_i_reconstructed = grid_to_place_with_nonlin(g_i_clean, Wpg, thresh=thresh)
    
    # Step 5: Place cell -> Sensory
    s_output = place_to_sensory(p_i_reconstructed, Wsp)
    
    stats_dict = {
        's_input': s,
        'p_i': p_i,
        'g_i_noisy': g_i_noisy,
        'g_i_clean': g_i_clean,
        'p_i_reconstructed': p_i_reconstructed,
        's_output': s_output,
    }
    
    return s_output, stats_dict


# ===== 准备数据：构建 premise 和 centroid 的向量映射 =====
print("\n1. 准备 Premise 和 Centroid 向量...")

# 加载位置信息
positions_path = os.path.join(RESULTS_DIR, 'positions.json')
with open(positions_path, 'r', encoding='utf-8') as f:
    positions_data = json.load(f)
premise_locs = positions_data.get('premise_locs', {})
print(f"   Loaded {len(premise_locs)} premise locations")

# 准备数据结构
premise_self_recon_data = []  # Premise自重建数据
centroid_self_recon_data = []  # Centroid自重建数据

# 遍历所有样本
for idx, sample_idx in tqdm(index_to_largest_sample.items(), desc="Preparing data"):
    meta = causal_metadata[sample_idx]
    askfor = meta['ASKFOR']
    
    # 构造位置key
    loc_key = f"{idx}_{askfor}"
    if loc_key not in premise_locs:
        continue
    
    pos_i = premise_locs[loc_key]
    x_i, y_i = pos_i[0], pos_i[1]
    
    # 检查位置是否在合法范围内
    if not (0 <= x_i < Npos and 0 <= y_i < Npos):
        continue
    
    # 获取 premise 向量 (训练100次)
    # askfor='cause' → premise是Effect (causal_B)
    # askfor='effect' → premise是Cause (causal_A)
    if askfor == 'cause':
        premise_vec = causal_B[sample_idx]
        centroid_vec = causal_A[sample_idx]  # Gemma推断的Cause centroid
    else:
        premise_vec = causal_A[sample_idx]
        centroid_vec = causal_B[sample_idx]  # Gemma推断的Effect centroid
    
    # 获取真实的pbook和gbook向量（用于误差计算）
    true_p = pbook[:, x_i, y_i]
    true_g = gbook[:, x_i, y_i]
    
    # 获取 cluster 大小（训练次数）
    if idx in index_to_largest_cluster:
        cluster_size = index_to_largest_cluster[idx]['largest_count']
    else:
        cluster_size = 1  # 默认
    
    # 添加到数据集
    premise_self_recon_data.append({
        'idx': idx,
        'sample_idx': sample_idx,
        'askfor': askfor,
        'pos': (x_i, y_i),
        's_input': premise_vec,
        'true_p': true_p,
        'true_g': true_g,
        'training_count': 100,  # Premise训练100次
    })
    
    centroid_self_recon_data.append({
        'idx': idx,
        'sample_idx': sample_idx,
        'askfor': askfor,
        'pos': (x_i, y_i),
        's_input': centroid_vec,
        'true_p': true_p,  # 注意：centroid使用相同的位置
        'true_g': true_g,
        'training_count': cluster_size,  # Centroid训练次数=cluster大小
    })

print(f"   Prepared {len(premise_self_recon_data)} premise samples")
print(f"   Prepared {len(centroid_self_recon_data)} centroid samples")


# ===== 对 Premise 进行自重建分析 =====
print("\n2. 对 Premise 进行自重建分析（训练100次）...")

premise_errors = {
    'err_p_i': [],
    'err_g_i_noisy': [],
    'err_g_i_clean': [],
    'err_p_reconstructed': [],
    'err_s_output': [],
}

for data in tqdm(premise_self_recon_data, desc="Premise self-reconstruction"):
    s_input = data['s_input']
    true_p = data['true_p']
    true_g = data['true_g']
    
    try:
        s_output, stats = self_reconstruction(
            s_input, Wps, Wsp, Wgp, Wpg, 
            module_gbooks, module_sizes, thresh=thresh
        )
        
        # 计算各步误差
        err_p_i = np.sum(np.abs(stats['p_i'] - true_p)) / (2 * Np)
        err_g_i_noisy = np.sum(np.abs(stats['g_i_noisy'] - true_g)) / (2 * Ng)
        err_g_i_clean = np.sum(np.abs(stats['g_i_clean'] - true_g)) / (2 * Ng)
        err_p_reconstructed = np.sum(np.abs(stats['p_i_reconstructed'] - true_p)) / (2 * Np)
        err_s_output = np.sum(np.abs(stats['s_output'] - s_input)) / (2 * Ns)  # 重建回原输入
        
        premise_errors['err_p_i'].append(err_p_i)
        premise_errors['err_g_i_noisy'].append(err_g_i_noisy)
        premise_errors['err_g_i_clean'].append(err_g_i_clean)
        premise_errors['err_p_reconstructed'].append(err_p_reconstructed)
        premise_errors['err_s_output'].append(err_s_output)
        
    except Exception as e:
        continue

print(f"   Successfully processed {len(premise_errors['err_p_i'])} premise samples")


# ===== 对 Centroid 进行自重建分析 =====
# test_mode: "gt" = 使用 ground truth g_i^clean，"org" = 使用 module_wise_NN_2d 清洗
test_mode = "org"  # 可选: "gt" 或 "org"

if test_mode == "gt":
    print("\n3. 对 Centroid 进行自重建分析（训练次数=cluster大小，g_i^clean 使用 ground truth）...")
else:
    print("\n3. 对 Centroid 进行自重建分析（训练次数=cluster大小，g_i^clean 使用 module_wise_NN_2d）...")

centroid_errors = {
    'err_p_i': [],
    'err_g_i_noisy': [],
    'err_g_i_clean': [],
    'err_p_reconstructed': [],
    'err_s_output': [],
    'training_count': [],  # 记录训练次数
}

for data in tqdm(centroid_self_recon_data, desc="Centroid self-reconstruction"):
    s_input = data['s_input']
    true_p = data['true_p']
    true_g = data['true_g']  # ground truth g_i
    training_count = data['training_count']
    
    try:
        # Step 1: Sensory -> Place cell
        s = normalize_vec(s_input)
        p_i = sensory_to_place(s, Wps)
        
        # Step 2: Place cell -> Grid cell (noisy)
        g_i_noisy = place_to_grid_noisy(p_i, Wgp)
        
        # Step 3: Grid cell 清洗
        if test_mode == "gt":
            # 使用 ground truth，避免 module_wise_NN_2d 拉回错误吸引子
            g_i_clean = true_g
            err_g_i_clean = 0.0
        else:
            # 使用原本的 module_wise_NN_2d 清洗
            gin = g_i_noisy.reshape(1, -1, 1)
            g_i_clean = module_wise_NN_2d(gin, module_gbooks, module_sizes)
            g_i_clean = g_i_clean[0, :, 0]
            err_g_i_clean = np.sum(np.abs(g_i_clean - true_g)) / (2 * Ng)
        
        # Step 4: Grid cell -> Place cell
        p_i_reconstructed = grid_to_place_with_nonlin(g_i_clean, Wpg, thresh=thresh)
        
        # Step 5: Place cell -> Sensory
        s_output = place_to_sensory(p_i_reconstructed, Wsp)
        
        # 计算各步误差
        err_p_i = np.sum(np.abs(p_i - true_p)) / (2 * Np)
        err_g_i_noisy = np.sum(np.abs(g_i_noisy - true_g)) / (2 * Ng)
        err_p_reconstructed = np.sum(np.abs(p_i_reconstructed - true_p)) / (2 * Np)
        err_s_output = np.sum(np.abs(s_output - s_input)) / (2 * Ns)  # 重建回原输入
        
        centroid_errors['err_p_i'].append(err_p_i)
        centroid_errors['err_g_i_noisy'].append(err_g_i_noisy)
        centroid_errors['err_g_i_clean'].append(err_g_i_clean)
        centroid_errors['err_p_reconstructed'].append(err_p_reconstructed)
        centroid_errors['err_s_output'].append(err_s_output)
        centroid_errors['training_count'].append(training_count)
        
    except Exception as e:
        continue

print(f"   Successfully processed {len(centroid_errors['err_p_i'])} centroid samples")
print(f"   Test mode: {test_mode}")


# ===== 打印统计结果 =====
print("\n" + "="*70)
print(f"Self-Reconstruction Error Statistics (test_mode={test_mode})")
print("="*70)

step_names = [
    ('err_p_i',           r's_i → p_i'),
    ('err_g_i_noisy',     r'p_i → g_i^noisy'),
    ('err_g_i_clean',     r'g_i^noisy → g_i^clean'),
    ('err_p_reconstructed', r'g_i^clean → p_i'),
    ('err_s_output',      r'p_i → s_i'),
]

print("\n--- Premise (训练100次) ---")
print(f"{'Step':<40} {'Mean Err':>12} {'Std':>12} {'Min':>12} {'Max':>12}")
print("-"*90)
for key, name in step_names:
    errs = np.array(premise_errors[key])
    print(f"{name:<40} {errs.mean():>12.4f} {errs.std():>12.4f} {errs.min():>12.4f} {errs.max():>12.4f}")

print(f"\n--- Centroid (训练次数=cluster大小, mode={test_mode}) ---")
print(f"{'Step':<40} {'Mean Err':>12} {'Std':>12} {'Min':>12} {'Max':>12}")
print("-"*90)
for key, name in step_names:
    errs = np.array(centroid_errors[key])
    print(f"{name:<40} {errs.mean():>12.4f} {errs.std():>12.4f} {errs.min():>12.4f} {errs.max():>12.4f}")

# 训练次数分布
training_counts = np.array(centroid_errors['training_count'])
print(f"\n--- Centroid 训练次数分布 ---")
print(f"  Mean: {training_counts.mean():.2f}")
print(f"  Std:  {training_counts.std():.2f}")
print(f"  Min:  {training_counts.min()}")
print(f"  Max:  {training_counts.max()}")
print(f"  Median: {np.median(training_counts):.0f}")

In [ ]:
# ===== Cell 9.2: 可视化 - Premise 和 Centroid 各步重建率 =====
"""
绘制两张图：
1. Premise 各步重建率（训练100次）
2. Centroid 各步重建率（训练次数=cluster大小）
"""

import matplotlib.pyplot as plt
import matplotlib as mpl

# Set global font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['legend.fontsize'] = 10

# ===== Figure 1: Premise Self-Reconstruction =====
fig1, ax1 = plt.subplots(figsize=(12, 6))

# Define step labels with LaTeX formatting for self-reconstruction
step_labels_self = [
    r'$s_i \rightarrow p_i$',
    r'$p_i \rightarrow g_i^{\mathrm{noisy}}$',
    r'$g_i^{\mathrm{noisy}} \rightarrow g_i^{\mathrm{clean}}$',
    r'$g_i^{\mathrm{clean}} \rightarrow p_i$',
    r'$p_i \rightarrow s_i$'
]

# Mean errors for each step (Premise)
premise_mean_errors = [
    np.mean(premise_errors['err_p_i']),
    np.mean(premise_errors['err_g_i_noisy']),
    np.mean(premise_errors['err_g_i_clean']),
    np.mean(premise_errors['err_p_reconstructed']),
    np.mean(premise_errors['err_s_output'])
]

# Color gradient (blue theme for Premise)
colors_premise = ['#1a237e', '#303f9f', '#3f51b5', '#7986cb', '#c5cae9']

# Create bar plot
x_pos = np.arange(len(step_labels_self))
bars = ax1.bar(x_pos, premise_mean_errors, color=colors_premise, edgecolor='white', linewidth=2, alpha=0.9)

# Add value labels on top of bars
for bar, val in zip(bars, premise_mean_errors):
    height = bar.get_height()
    if height > 0.001:
        ax1.annotate(f'{val:.4f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=11, fontweight='bold')
    else:
        ax1.annotate(f'{val:.6f}',
                    xy=(bar.get_x() + bar.get_width() / 2, 0.002),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10, color='gray')

# Styling
ax1.set_xlabel('Self-Reconstruction Step', fontsize=13, fontweight='bold')
ax1.set_ylabel('Mean Reconstruction Error', fontsize=13, fontweight='bold')
ax1.set_title('Premise Self-Reconstruction Error (Training: 100 times)', fontsize=15, fontweight='bold', pad=15)
ax1.set_xticks(x_pos)
ax1.set_xticklabels(step_labels_self, fontsize=12)
ax1.set_ylim(0, max(premise_mean_errors) * 1.3)

# Add grid
ax1.grid(axis='y', alpha=0.3, linestyle='--', zorder=0)
ax1.set_axisbelow(True)

# Remove top and right spines
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

# Add annotation about error definition
ax1.annotate(r'Error: $\frac{\sum |predicted - true|}{2N}$', 
            xy=(0.98, 0.95), xycoords='axes fraction',
            ha='right', va='top', fontsize=11,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='whitesmoke', edgecolor='gray', alpha=0.8))

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, 'premise_self_reconstruction.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✅ Figure 1: Premise Self-Reconstruction saved")


# ===== Figure 2: Centroid Self-Reconstruction =====
fig2, ax2 = plt.subplots(figsize=(12, 6))

# Mean errors for each step (Centroid)
centroid_mean_errors = [
    np.mean(centroid_errors['err_p_i']),
    np.mean(centroid_errors['err_g_i_noisy']),
    np.mean(centroid_errors['err_g_i_clean']),
    np.mean(centroid_errors['err_p_reconstructed']),
    np.mean(centroid_errors['err_s_output'])
]

# Color gradient (orange theme for Centroid)
colors_centroid = ['#e65100', '#f57c00', '#ff9800', '#ffb74d', '#ffe0b2']

# Create bar plot
bars = ax2.bar(x_pos, centroid_mean_errors, color=colors_centroid, edgecolor='white', linewidth=2, alpha=0.9)

# Add value labels on top of bars
for bar, val in zip(bars, centroid_mean_errors):
    height = bar.get_height()
    if height > 0.001:
        ax2.annotate(f'{val:.4f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=11, fontweight='bold')
    else:
        ax2.annotate(f'{val:.6f}',
                    xy=(bar.get_x() + bar.get_width() / 2, 0.002),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10, color='gray')

# Calculate mean training count
mean_training_count = np.mean(centroid_errors['training_count'])

# Styling
ax2.set_xlabel('Self-Reconstruction Step', fontsize=13, fontweight='bold')
ax2.set_ylabel('Mean Reconstruction Error', fontsize=13, fontweight='bold')
ax2.set_title(f'Centroid Self-Reconstruction Error (Mean Training: {mean_training_count:.1f} times)', 
              fontsize=15, fontweight='bold', pad=15)
ax2.set_xticks(x_pos)
ax2.set_xticklabels(step_labels_self, fontsize=12)
ax2.set_ylim(0, max(centroid_mean_errors) * 1.3)

# Add grid
ax2.grid(axis='y', alpha=0.3, linestyle='--', zorder=0)
ax2.set_axisbelow(True)

# Remove top and right spines
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

# Add annotation about error definition
ax2.annotate(r'Error: $\frac{\sum |predicted - true|}{2N}$', 
            xy=(0.98, 0.95), xycoords='axes fraction',
            ha='right', va='top', fontsize=11,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='whitesmoke', edgecolor='gray', alpha=0.8))

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, 'centroid_self_reconstruction.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✅ Figure 2: Centroid Self-Reconstruction saved")

In [ ]:
# ===== Cell 9.3: 可视化 - 重建率与训练次数的关系 =====
"""
绘制 Centroid 各步重建率随训练次数（cluster大小）的变化曲线
只关注 s-p, p-g, p-s 三步

分析重建率是否与训练次数相关
"""

import scipy.stats as stats

# ===== 准备数据：按训练次数分组 =====
training_counts = np.array(centroid_errors['training_count'])
unique_counts = np.unique(training_counts)

# 计算每个训练次数对应的平均误差（只关注3个步骤）
count_to_errors = {step: {} for step in ['err_p_i', 'err_g_i_noisy', 'err_s_output']}

for step_key in count_to_errors.keys():
    step_errs = np.array(centroid_errors[step_key])
    for count in unique_counts:
        mask = training_counts == count
        count_to_errors[step_key][count] = step_errs[mask].mean()

# ===== Figure 3: 重建率随训练次数的变化 (只关注 s-p, p-g, p-s) =====
fig3, axes = plt.subplots(1, 3, figsize=(15, 5))

step_keys = ['err_p_i', 'err_g_i_noisy', 'err_s_output']
step_titles = [
    r'$s_i \rightarrow p_i$',
    r'$p_i \rightarrow g_i^{\mathrm{noisy}}$',
    r'$p_i \rightarrow s_i$'
]

colors_scatter = ['#1976d2', '#388e3c', '#7b1fa2']

for i, (step_key, step_title) in enumerate(zip(step_keys, step_titles)):
    ax = axes[i]
    
    # 获取数据
    step_errs = np.array(centroid_errors[step_key])
    
    # 散点图
    ax.scatter(training_counts, step_errs, alpha=0.4, s=30, c=colors_scatter[i], edgecolors='white', linewidth=0.5)
    
    # 按训练次数分组计算均值
    sorted_counts = sorted(unique_counts)
    mean_errs = [count_to_errors[step_key][c] for c in sorted_counts]
    
    # 绘制趋势线（均值折线）
    ax.plot(sorted_counts, mean_errs, 'o-', color='black', linewidth=2, markersize=6, 
            label='Mean per count', zorder=5)
    
    # 计算相关系数
    correlation, p_value = stats.pearsonr(training_counts, step_errs)
    
    # 设置标签和标题
    ax.set_xlabel('Training Count (Cluster Size)', fontsize=11)
    ax.set_ylabel('Reconstruction Error', fontsize=11)
    ax.set_title(step_title, fontsize=13, fontweight='bold')
    
    # 添加相关系数标注
    ax.annotate(f'Pearson r = {correlation:.4f}\np-value = {p_value:.2e}', 
                xy=(0.95, 0.95), xycoords='axes fraction',
                ha='right', va='top', fontsize=10,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', edgecolor='gray', alpha=0.8))
    
    # Grid
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# 整体标题
fig3.suptitle('Centroid Self-Reconstruction Error vs Training Count (Cluster Size)', 
              fontsize=16, fontweight='bold', y=1.02)

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, 'reconstruction_vs_training_count.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✅ Figure 3: Reconstruction Error vs Training Count saved")


# ===== 汇总相关性分析 =====
print("\n" + "="*70)
print("Correlation Analysis: Reconstruction Error vs Training Count")
print("="*70)
print(f"{'Step':<45} {'Pearson r':>12} {'p-value':>15} {'Significant':>15}")
print("-"*90)

for step_key, step_title in zip(step_keys, step_titles):
    step_errs = np.array(centroid_errors[step_key])
    corr, pval = stats.pearsonr(training_counts, step_errs)
    significant = "Yes (p<0.05)" if pval < 0.05 else "No"
    print(f"{step_title:<45} {corr:>12.4f} {pval:>15.2e} {significant:>15}")

# 最终重建误差 (s_output) 的详细分析
final_errs = np.array(centroid_errors['err_s_output'])
print(f"\n--- 最终重建误差 (s_i → s_i) 与训练次数的关系 ---")
print(f"  Pearson correlation: {stats.pearsonr(training_counts, final_errs)[0]:.4f}")
print(f"  Spearman correlation: {stats.spearmanr(training_counts, final_errs)[0]:.4f}")

# 按训练次数分组的统计
print(f"\n--- 按训练次数分组的最终重建误差 ---")
bins = [1, 5, 10, 20, 50, 100, np.inf]
bin_labels = ['1-4', '5-9', '10-19', '20-49', '50-99', '100+']
for i in range(len(bins)-1):
    mask = (training_counts >= bins[i]) & (training_counts < bins[i+1])
    if mask.sum() > 0:
        mean_err = final_errs[mask].mean()
        std_err = final_errs[mask].std()
        count = mask.sum()
        print(f"  Count {bin_labels[i]:>6}: Mean={mean_err:.4f}, Std={std_err:.4f}, N={count}")

In [ ]:
# ===== Cell 9.4: 对比图 - Premise vs Centroid 重建率 =====
"""
绘制对比图：Premise vs Centroid 各步重建率
"""

from datetime import datetime
import scipy.stats as scipy_stats

# ===== Figure 4: Premise vs Centroid Comparison =====
fig4, ax = plt.subplots(figsize=(14, 7))

# Define step labels
step_labels_self = [
    r'$s_i \rightarrow p_i$',
    r'$p_i \rightarrow g_i^{\mathrm{noisy}}$',
    r'$g_i^{\mathrm{noisy}} \rightarrow g_i^{\mathrm{clean}}$',
    r'$g_i^{\mathrm{clean}} \rightarrow p_i$',
    r'$p_i \rightarrow s_i$'
]

# Mean errors
premise_mean_errors = [
    np.mean(premise_errors['err_p_i']),
    np.mean(premise_errors['err_g_i_noisy']),
    np.mean(premise_errors['err_g_i_clean']),
    np.mean(premise_errors['err_p_reconstructed']),
    np.mean(premise_errors['err_s_output'])
]

centroid_mean_errors = [
    np.mean(centroid_errors['err_p_i']),
    np.mean(centroid_errors['err_g_i_noisy']),
    np.mean(centroid_errors['err_g_i_clean']),
    np.mean(centroid_errors['err_p_reconstructed']),
    np.mean(centroid_errors['err_s_output'])
]

# Bar positions
x_pos = np.arange(len(step_labels_self))
width = 0.35

# Create grouped bar chart
bars1 = ax.bar(x_pos - width/2, premise_mean_errors, width, label='Premise (100 training)', 
               color='#1976d2', edgecolor='white', linewidth=1.5, alpha=0.9)
bars2 = ax.bar(x_pos + width/2, centroid_mean_errors, width, label=f'Centroid (mean {mean_training_count:.1f} training)', 
               color='#ff7043', edgecolor='white', linewidth=1.5, alpha=0.9)

# Add value labels
def add_labels(bars, vals, offset_y=5):
    for bar, val in zip(bars, vals):
        height = bar.get_height()
        if height > 0.001:
            ax.annotate(f'{val:.4f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, offset_y),
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=9, fontweight='bold')

add_labels(bars1, premise_mean_errors)
add_labels(bars2, centroid_mean_errors)

# Styling
ax.set_xlabel('Self-Reconstruction Step', fontsize=13, fontweight='bold')
ax.set_ylabel('Mean Reconstruction Error', fontsize=13, fontweight='bold')
ax.set_title('Premise vs Centroid: Self-Reconstruction Error Comparison', fontsize=15, fontweight='bold', pad=15)
ax.set_xticks(x_pos)
ax.set_xticklabels(step_labels_self, fontsize=11)
ax.set_ylim(0, max(max(premise_mean_errors), max(centroid_mean_errors)) * 1.35)

ax.legend(loc='upper right', fontsize=11, framealpha=0.9)
ax.grid(axis='y', alpha=0.3, linestyle='--', zorder=0)
ax.set_axisbelow(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add annotation
ax.annotate(r'Error: $\frac{\sum |predicted - true|}{2N}$', 
            xy=(0.02, 0.95), xycoords='axes fraction',
            ha='left', va='top', fontsize=11,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='whitesmoke', edgecolor='gray', alpha=0.8))

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, 'premise_vs_centroid_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✅ Figure 4: Premise vs Centroid Comparison saved")


# ===== 保存分析结果 =====
self_recon_results = {
    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'premise_analysis': {
        'training_count': 100,
        'sample_count': len(premise_errors['err_p_i']),
        'step_errors': {
            's_i → p_i': float(np.mean(premise_errors['err_p_i'])),
            'p_i → g_i^noisy': float(np.mean(premise_errors['err_g_i_noisy'])),
            'g_i^noisy → g_i^clean': float(np.mean(premise_errors['err_g_i_clean'])),
            'g_i^clean → p_i': float(np.mean(premise_errors['err_p_reconstructed'])),
            'p_i → s_i': float(np.mean(premise_errors['err_s_output'])),
        }
    },
    'centroid_analysis': {
        'mean_training_count': float(np.mean(centroid_errors['training_count'])),
        'sample_count': len(centroid_errors['err_p_i']),
        'step_errors': {
            's_i → p_i': float(np.mean(centroid_errors['err_p_i'])),
            'p_i → g_i^noisy': float(np.mean(centroid_errors['err_g_i_noisy'])),
            'g_i^noisy → g_i^clean': float(np.mean(centroid_errors['err_g_i_clean'])),
            'g_i^clean → p_i': float(np.mean(centroid_errors['err_p_reconstructed'])),
            'p_i → s_i': float(np.mean(centroid_errors['err_s_output'])),
        },
        'correlation_with_training_count': {
            step_title: {
                'pearson_r': float(scipy_stats.pearsonr(training_counts, np.array(centroid_errors[step_key]))[0]),
                'p_value': float(scipy_stats.pearsonr(training_counts, np.array(centroid_errors[step_key]))[1])
            }
            for step_key, step_title in zip(step_keys, step_titles)
        }
    }
}

# Save to JSON
self_recon_path = os.path.join(RESULTS_DIR, 'self_reconstruction_analysis.json')
with open(self_recon_path, 'w', encoding='utf-8') as f:
    json.dump(self_recon_results, f, ensure_ascii=False, indent=2)
print(f"\n✅ Self-reconstruction analysis saved to: {self_recon_path}")


# ===== 最终总结 =====
print("\n" + "="*70)
print("SELF-RECONSTRUCTION ANALYSIS SUMMARY")
print("="*70)
print(f"""
流程: s_i → p_i → g_i^noisy → g_i^clean → p_i → s_i (无路径积分)

1. Premise (训练100次):
   - 样本数: {len(premise_errors['err_p_i'])}
   - 最终重建误差 (p_i → s_i): {np.mean(premise_errors['err_s_output']):.4f}

2. Centroid (平均训练{np.mean(centroid_errors['training_count']):.1f}次):
   - 样本数: {len(centroid_errors['err_p_i'])}
   - 最终重建误差 (p_i → s_i): {np.mean(centroid_errors['err_s_output']):.4f}

3. 误差差异:
   - Premise 最终误差: {np.mean(premise_errors['err_s_output']):.4f}
   - Centroid 最终误差: {np.mean(centroid_errors['err_s_output']):.4f}
   - 差值 (Centroid - Premise): {np.mean(centroid_errors['err_s_output']) - np.mean(premise_errors['err_s_output']):.4f}

4. 训练次数与重建误差的相关性:
   - 最终误差 (s_i → s_i) 的 Pearson r: {scipy_stats.pearsonr(training_counts, np.array(centroid_errors['err_s_output']))[0]:.4f}

   - p-value: {scipy_stats.pearsonr(training_counts, np.array(centroid_errors['err_s_output']))[1]:.2e}""")